In [10]:
import numpy as np

def thermodynamic_constants(compounds):
    properties = {
        "water": {"antoine": [5.11564, 1687.537, 230.17], 
                   "Cp_liq": 76.392/1000, "Cp_vap": 33.59/1000, "Hvap": 35.003, "activity coefficient":1}, #Hvap is at Tb: 373.15 K
        "nitric acid": {"antoine": [4.28523, 1337.035, 224.78], 
                         "Cp_liq": 110.679/1000, "Cp_vap": 53.336/1000, "Hvap": 25.285, "activity coefficient":0.725}, #Hvap is at Tb: 356.15 K
        "ammonium nitrate": {"antoine": [7.105, 4360, 0], 
                         "Cp_liq": 132.2144/1000, "Cp_vap": 85.353/1000, "Hvap": 50, "activity coefficient":0.142} #Hvap is at Tb: 356.15 K
        
    }
        
    cts_antoine = np.array([properties[comp]["antoine"] for comp in compounds])
    Cp_liq = np.array([properties[comp]["Cp_liq"] for comp in compounds])
    Cp_vap = np.array([properties[comp]["Cp_vap"] for comp in compounds])
    Hvap = np.array([properties[comp]["Hvap"] for comp in compounds])
    Activity_coefficient = np.array([properties[comp]["activity coefficient"] for comp in compounds])
 
    return (cts_antoine, Cp_liq, Cp_vap, Hvap, Activity_coefficient)

In [11]:
def pure_vapor_pressure(compounds, T):
    
    cts, _ , _, _ , _ = thermodynamic_constants(compounds)
    p_vap = np.zeros(len(compounds))
    for i in range(len(compounds)):
        A = cts[i][0]
        B = cts[i][1]
        C = cts[i][2]
        
        exp_vap = A - (B / (T + C - 273.15)) #antoine equation
        p_vap[i] = pow(10, exp_vap)
        
    return p_vap

In [12]:

def Bubble_pressure(compounds, T, x):
    
    Pvap_mix = pure_vapor_pressure(compounds, T)
    
    
    # Extract activity coefficients
    _ , _ , _ , _ , activity_coefficients = thermodynamic_constants(compounds)
    activity_coefficients = activity_coefficients.flatten()  # Convert to 1D array
    
    tol = 1.0e-10

    Po = sum(x * Pvap_mix)

    fequil = 1.0

    while abs(fequil) > tol:
        Ki = (activity_coefficients * Pvap_mix) / Po  #modified raoults law  
        y = Ki * x  
        fequil = 1.0 - sum(Ki * x)  

        defun = 10*sum(Ki * x) / Po  
        P1 = Po - (fequil / defun)
        Po = P1
    
    return P1, y


In [13]:
compounds = ["water","nitric acid"]
Z = [0.985290088,0.014709912]
T = 118.1 + 273.15 #C

print(Bubble_pressure(compounds, T, Z))

(1.8612786810372681, array([0.98607032, 0.01392968]))


In [16]:
def Dew_pressure(compounds, T, y):
    
    Pvap_mix = pure_vapor_pressure(compounds, T)
    
    
    # Extract activity coefficients
    _ , _ , _ , _ , activity_coefficients = thermodynamic_constants(compounds)
    activity_coefficients = activity_coefficients.flatten()  # Convert to 1D array
    
    tol = 1.0e-10

    Po = sum(y * Pvap_mix)

    fequil = 1.0

    while abs(fequil) > tol:
        Ki = (activity_coefficients * Pvap_mix) / Po  #modified raoults law 
        Ki = np.maximum(Ki, 1e-10)
        x = y / Ki  
        fequil = 1.0 - sum(y / Ki)  

        defun = 10*sum(y / Ki) / Po  
        P1 = Po - (fequil / defun)
        Po = P1
    
    return P1, x

In [17]:
compounds = ["water","nitric acid"]
Z = [0.985290088,0.014709912]
T = 118.1 + 273.15 #C

print(Dew_pressure(compounds, T, Z))

/var/folders/s0/7945scyx493_5qvr8prl0z6r0000gn/T/ipykernel_7146/1663351390.py:23: RuntimeWarning: overflow encountered in double_scalars
  P1 = Po - (fequil / defun)
/var/folders/s0/7945scyx493_5qvr8prl0z6r0000gn/T/ipykernel_7146/1663351390.py:23: RuntimeWarning: divide by zero encountered in double_scalars
  P1 = Po - (fequil / defun)


KeyboardInterrupt: 